In [1]:
import pandas as pd 
import openpyxl 
from shapely.geometry import Point, Polygon
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import re

In [79]:
# 
workbook =pd.read_excel('../data/landing/Book2.xlsx')

pop_column = workbook['SA2_code_population ']
wealth_column= workbook['SA2_code_wealth']
suburb_column = workbook["suburb"]

workbook_wealth = openpyxl.load_workbook('../data/landing/new_wealth/new_wealth_data.xlsx')
sheet=workbook_wealth["Table 1.4"]
matching_rows = []
pattern = re.compile(r'2\d{8}$')
row_iterator= sheet.iter_rows(values_only=True)
for i in range(8):
    next (row_iterator)
for row  in row_iterator:
    code_value = str(row[0])
    if pattern.match(code_value):
        matching_rows.append(row)
value_list=[]
for i in matching_rows:
    value_list.append([i[0],i[17],i[18],i[19],i[20],i[21]]) 
    column_names = ["Sa2_code", "median_2015_2016","median_2016_2017","median_2017_2018","median_2018_2019","median_2019_2020"]
df = pd.DataFrame(value_list, columns=column_names)

In [84]:
workbook_wealth = openpyxl.load_workbook('../data/landing/new_wealth/new_wealth_data.xlsx')
sheet=workbook_wealth["Table 1.4"]
matching_rows = []
pattern = re.compile(r'2\d{8}$')
row_iterator= sheet.iter_rows(values_only=True)
for i in range(8):
    next (row_iterator)
for row  in row_iterator:
    code_value = str(row[0])
    if pattern.match(code_value):
        matching_rows.append(row)
value_list=[]
for i in matching_rows:
    value_list.append([i[0],i[17],i[18],i[19],i[20],i[21]]) 
    column_names = ["Sa2_code", "median_2015_2016","median_2016_2017","median_2017_2018","median_2018_2019","median_2019_2020"]
df = pd.DataFrame(value_list, columns=column_names)
median_2015_2016_list =[]
median_2016_2017_list=[]
median_2017_2018_list=[]
median_2018_2019_list=[]
median_2019_2020_list=[]
for i in wealth_column:
    # is not emoty 
    if not pd.isna(i):
        # more than one sa2_code in the box 
        if type(i)==str:
            total_median_2015_2016=0
            total_median_2016_2017=0
            total_median_2017_2018=0
            total_median_2018_2019=0
            total_median_2019_2020=0
            number =0 
            for each_code in i.split(","):
                pattern = r"(?:None|\d+)"
                sa2_code=each_code
                matches = re.findall(pattern, sa2_code)
                matched=False 
                
                
                for value_count in range(len(df)):
                    value_sa2_code = df["Sa2_code"][value_count]
                    if str(matches[0])==str(value_sa2_code):
                        matched= True
                        total_median_2015_2016+=df["median_2015_2016"][value_count]
                        total_median_2016_2017+=df["median_2016_2017"][value_count]
                        total_median_2017_2018+=df["median_2017_2018"][value_count]
                        total_median_2018_2019+=df["median_2018_2019"][value_count]
                        total_median_2019_2020+=df["median_2019_2020"][value_count]
                
                number+=1   
                if not matched:
                    print("failed")
                    print(sa2_code)
            median_2015_2016_list.append(float(total_median_2015_2016)/number)
            median_2016_2017_list.append(float(total_median_2016_2017)/number)
            median_2017_2018_list.append(float(total_median_2017_2018)/number)
            median_2018_2019_list.append(float(total_median_2018_2019)/number)
            median_2019_2020_list.append(float(total_median_2019_2020)/number)
                
                    
                
        else:
            sa2_code =i
            matched=False 
            for value_count in range(len(df)):
                value_sa2_code = df["Sa2_code"][value_count]
                if str(sa2_code)==str(value_sa2_code):
                    matched= True
                    median_2015_2016_list.append(df["median_2015_2016"][value_count])
                    median_2016_2017_list.append(df["median_2016_2017"][value_count])
                    median_2017_2018_list.append(df["median_2017_2018"][value_count])
                    median_2018_2019_list.append(df["median_2018_2019"][value_count])
                    median_2019_2020_list.append(df["median_2019_2020"][value_count])
                
            if not matched:
                print("failed")
                print(sa2_code)
data = {'suburb': suburb_column, 'median_2015_2016': median_2015_2016_list,'median_2016_2017': median_2016_2017_list,
        'median_2017_2018': median_2017_2018_list,'median_2018_2019': median_2018_2019_list,'median_2019_2020': median_2019_2020_list}
wealth_df = pd.DataFrame(data)
wealth_df.to_csv("../data/raw/time_series_wealth.csv")

In [86]:
geopackage_path = "../data/landing/Geopackage/SA2 ERP GeoPackage 2022.gpkg"
gdf = gpd.read_file(geopackage_path)
selected_row = gdf[gdf["State_name_2021"]=="Victoria"]
selected_row = selected_row.reset_index(drop=True)
ERP_2022_list =[]
ERP_2021_list=[]
ERP_2020_list=[]
ERP_2019_list=[]
ERP_2018_list=[]
for i in pop_column:
    # is not emoty 
    if not pd.isna(i):
        # more than one sa2_code in the box 
        if type(i)==str:
            total_2018_pop=0
            total_2019_pop=0
            total_2020_pop=0
            total_2021_pop=0
            total_2022_pop=0
            number =0 
            for each_code in i.split(","):
                pattern = r"(?:None|\d+)"
                sa2_code=each_code
                matches = re.findall(pattern, sa2_code)
                matched=False 
                for value_count in range(len(selected_row)):
                    value_sa2_code = selected_row["SA2_code_2021"][value_count]
                    if str(matches[0])==str(value_sa2_code):
                        matched= True
                        total_2018_pop+=selected_row["ERP_2018"][value_count]
                        total_2019_pop+=selected_row["ERP_2019"][value_count]
                        total_2020_pop+=selected_row["ERP_2020"][value_count]
                        total_2021_pop+=selected_row["ERP_2021"][value_count]
                        total_2022_pop+=selected_row["ERP_2022"][value_count]
                number+=1   
                if not matched:
            ERP_2022_list.append(float(total_2022_pop)/number)
            ERP_2021_list.append(float(total_2021_pop)/number)
            ERP_2020_list.append(float(total_2020_pop)/number)
            ERP_2019_list.append(float(total_2019_pop)/number)
            ERP_2018_list.append(float(total_2018_pop)/number)
        
        else:
            sa2_code =i
            matched=False 
            for value_count in range(len(selected_row)):
                value_sa2_code = selected_row["SA2_code_2021"][value_count]
                if str(sa2_code)==str(value_sa2_code):
                    matched= True
                    ERP_2022_list.append(selected_row["ERP_2022"][value_count])
                    ERP_2021_list.append(selected_row["ERP_2021"][value_count])
                    ERP_2020_list.append(selected_row["ERP_2020"][value_count])
                    ERP_2019_list.append(selected_row["ERP_2019"][value_count])
                    ERP_2018_list.append(selected_row["ERP_2018"][value_count])
                
            if not matched:
data = {'suburb': suburb_column, 'ERP_2018': ERP_2018_list,'ERP_2019': ERP_2019_list,
        'ERP_2020': ERP_2020_list,'ERP_2021': ERP_2021_list,'ERP_2022': ERP_2022_list}
wealth_df = pd.DataFrame(data)
wealth_df.to_csv("../data/raw/time_series_population.csv")

3
2
2
3
2
2
2
2
2
3
2
2
3
3
2
2
2
2
3
2
2
2
